In [1]:
import flash
import mlflow
import torch
from flash.core.data.utils import download_data
from flash.text import TextClassificationData, TextClassifier
import torchmetrics
from pytorch_lightning.core.memory import ModelSummary
import shutil
import tempfile
import os


In [2]:
def save_file_as_mlflow_artifact(file_content_string, file_name = 'model_summary.txt'):
    tempdir = tempfile.mkdtemp()
    try:
        summary_file = os.path.join(tempdir, file_name)
        with open(summary_file, "w") as f:
            f.write(file_content_string)

        mlflow.log_artifact(local_path=summary_file)
    finally:
        shutil.rmtree(tempdir)

In [3]:
download_data("https://pl-flash-data.s3.amazonaws.com/imdb.zip", "./data/")
datamodule = TextClassificationData.from_csv(
    input_fields="review",
    target_fields="sentiment",
    train_file="data/imdb/train.csv",
    val_file="data/imdb/valid.csv",
    test_file="data/imdb/test.csv"
)

/Users/yongliu/opt/miniconda3/envs/dl_model/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'pl-flash-data.s3.amazonaws.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
Using custom data configuration default


 15%|█▌        | 3382/22500 [00:00<00:00, 33819.39ex/s]

Dataset csv downloaded and prepared to /Users/yongliu/.cache/huggingface/datasets/csv/default-6a1d0b9ab3bf3352/0.0.0/2960f95a26e85d40ca41a230ac88787f715ee3003edaacb8b1f0891e9f04dda2. Subsequent calls will reuse this data.


100%|██████████| 23/23 [00:05<00:00,  4.04ba/s]
Using custom data configuration default
  0%|          | 0/2500 [00:00<?, ?ex/s]

Dataset csv downloaded and prepared to /Users/yongliu/.cache/huggingface/datasets/csv/default-b28eab8918e65374/0.0.0/2960f95a26e85d40ca41a230ac88787f715ee3003edaacb8b1f0891e9f04dda2. Subsequent calls will reuse this data.


100%|██████████| 3/3 [00:00<00:00,  4.45ba/s]
Using custom data configuration default


Dataset csv downloaded and prepared to /Users/yongliu/.cache/huggingface/datasets/csv/default-139b07428d91ba40/0.0.0/2960f95a26e85d40ca41a230ac88787f715ee3003edaacb8b1f0891e9f04dda2. Subsequent calls will reuse this data.


100%|██████████| 3/3 [00:00<00:00,  4.37ba/s]


In [4]:
classifier_model = TextClassifier(backbone="prajjwal1/bert-tiny", num_classes=datamodule.num_classes, metrics=torchmetrics.F1(datamodule.num_classes))
trainer = flash.Trainer(max_epochs=3, gpus=torch.cuda.device_count())

Using 'prajjwal1/bert-tiny' provided by Hugging Face/transformers (https://github.com/huggingface/transformers).
Some weights of the model checkpoint at prajjwal1/bert-tiny were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification

In [6]:

EXPERIMENT_NAME = "dl_model_chapter03"
mlflow.set_tracking_uri('http://localhost')
mlflow.set_experiment(EXPERIMENT_NAME)
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
print("experiment_id:", experiment.experiment_id)

experiment_id: 1


In [7]:
MODEL_ARTIFACT_PATH = 'dl_model'
REGISTERED_MODEL_NAME = 'nlp_dl_model'
with mlflow.start_run(experiment_id=experiment.experiment_id, run_name="chapter03") as dl_model_tracking_run:
    trainer.finetune(classifier_model, datamodule=datamodule, strategy="freeze")
    trainer.test()

    # mlflow log metrics
    cur_metrics = trainer.callback_metrics
    # Cast metric value as  float before passing into logger.
    metrics = dict(map(lambda x: (x[0], float(x[1])), cur_metrics.items()))
    mlflow.log_metrics(metrics)

    # mlflow log parameters
    params = {"epochs": trainer.max_epochs}
    if hasattr(trainer, "optimizers"):
        optimizer = trainer.optimizers[0]
        params["optimizer_name"] = optimizer.__class__.__name__
    if hasattr(optimizer, "defaults"):
        params.update(optimizer.defaults)
    # add hyper-parameters used in this training
    params.update(classifier_model.hparams)
    mlflow.log_params(params)

    # log model summary as an artifact
    summary = ModelSummary(classifier_model, max_depth=-1)
    save_file_as_mlflow_artifact(str(summary))

    # log and register the trained model
    mlflow.pytorch.log_model(pytorch_model=classifier_model, artifact_path=MODEL_ARTIFACT_PATH, registered_model_name=REGISTERED_MODEL_NAME)



  | Name          | Type                          | Params
----------------------------------------------------------------
0 | train_metrics | ModuleDict                    | 0     
1 | val_metrics   | ModuleDict                    | 0     
2 | model         | BertForSequenceClassification | 4.4 M 
----------------------------------------------------------------
258       Trainable params
4.4 M     Non-trainable params
4.4 M     Total params
17.545    Total estimated model params size (MB)


Testing: 100%|█████████▉| 622/625 [00:10<00:00, 59.06it/s]--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_cross_entropy': 0.785443127155304, 'test_f1': 0.5343999862670898}
--------------------------------------------------------------------------------
Testing: 100%|██████████| 625/625 [00:10<00:00, 58.59it/s]


Registered model 'nlp_dl_model' already exists. Creating a new version of this model...
2021/10/24 15:44:35 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: nlp_dl_model, version 6
Created version '6' of model 'nlp_dl_model'.


In [8]:
run_id = dl_model_tracking_run.info.run_id
print("run_id: {}; lifecycle_stage: {}".format(run_id,
    mlflow.get_run(run_id).info.lifecycle_stage))

run_id: 7ccfa0a359ca4cae953a90d7d5a59c4b; lifecycle_stage: active


In [50]:
logged_model = f'runs:/{run_id}/{MODEL_ARTIFACT_PATH}'
# Load model as a pytorch model, not as the pyfunc model
model = mlflow.pytorch.load_model(logged_model)
# To bypass a lightning-flash's bug, we need to set the stage to test so a loaded model can be used to do prediction
model.trainer.state.stage='test'
model.predict({'This is great news'})

['positive']